In [1]:
from pathlib import Path
from matplotlib import pyplot as plt
from PIL import Image, ImageEnhance, ImageOps, ImageChops, ImageDraw, ImageFilter, ImageTransform
import random
import shutil
import numpy as np
import cv2
import glob
import os

In [2]:
### Source of the hiragana images
source = Path('C:/Users/andma/OneDrive/Documenti/hiragana images/hiragana_images')
# Set the source and destination directories
src_dir = r"C:\Users\andma\OneDrive\Documenti\hiragana images\hiragana_images_original\hiragana_images"
dst_dir = r"C:\Users\andma\OneDrive\Documenti\hiragana images\hiragana_images"

In [8]:
# Delete all the images in the folders contained in dst_dir
for folder in os.listdir(dst_dir):
    folder_path = os.path.join(dst_dir, folder)
    for file in os.listdir(folder_path):
        file_path = os.path.join(folder_path, file)
        os.remove(file_path)

# Copy all the images from src_dir to dst_dir
for file in os.listdir(src_dir):
    src_file = os.path.join(src_dir, file)
    name = ""
    for i in range(4, 7):
        if file[i].isdigit() == False:
            name = name + file[i]
    dst_folder_name = name  # get the letters after "kana"
    dst_folder = os.path.join(dst_dir, dst_folder_name)
    dst_file = os.path.join(dst_folder, file)
    shutil.copy(src_file, dst_folder)

In [9]:
### Obtaining the complete paths of all source images
subdirect = [x for x in source.iterdir() if x.is_dir()]

subdirect_string = [str(path) for path in subdirect]

source_images = [glob.glob(path + '/*.jpg') for path in subdirect_string]

images_path = []
for lists in source_images:
    for path in lists:
        images_path.append(path)

In [10]:
subdirect = [x for x in source.iterdir() if x.is_dir()]

subdirect_string = [str(path) for path in subdirect]

for subfolder in subdirect_string:
    
    source_images = [glob.glob(subfolder + '/*.jpg')]  
    images_path = []
    for lists in source_images:
        for path in lists:
            images_path.append(path)
        
    for image in images_path:
        
        kana = cv2.imread(image) 
    
        #### Adding 1 pixel padding to the bottom of the images to obtain 84x84 pixels images
        padded_image = cv2.copyMakeBorder(kana, 0, 0, 1, 0, cv2.BORDER_CONSTANT)
        
        cv2.imwrite(image,padded_image) 

In [11]:
### Do I really need this??
# Set the desired resolution
desired_resolution = (84, 84)

# Iterate through each subdirectory in the root directory
for directory in os.listdir(dst_dir):
    directory_path = os.path.join(dst_dir, directory)
    # Check if the subdirectory is a directory
    if os.path.isdir(directory_path):
        # Iterate through each image file in the subdirectory
        for file in os.listdir(directory_path):
            file_path = os.path.join(directory_path, file)
            # Check if the file is an image
            if file.endswith(".png") or file.endswith(".jpg") or file.endswith(".jpeg"):
                # Open the image
                image = Image.open(file_path)
                # Crop the top of the image to the desired resolution
                image = image.crop((0, 0, desired_resolution[0], desired_resolution[1]))
                # Save the modified image
                image.save(file_path)

In [12]:
for subdir, dirs, files in os.walk(dst_dir):
    for file in files:
        file_path = os.path.join(subdir, file)
        image = Image.open(file_path)
        
        # Perform data augmentation
        angle = random.uniform(-15, 15)
        trans_x = random.uniform(-10, 10)
        trans_y = random.uniform(-10, 10)
        scale = random.uniform(0.8, 1.1)

        image = image.rotate(angle, resample=Image.Resampling.NEAREST)
        #image = image.transform(image.size, Image.Transform().AFFINE, (1, 0, trans_x, 0, 1, trans_y))
        image = image.resize((int(image.size[0]*scale), int(image.size[1]*scale)), resample=Image.Resampling.NEAREST)

        # Save augmented image with a different name
        new_file_name = file.split(".")[0] + "_aug.jpg"
        image.save(os.path.join(subdir, new_file_name))